In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession


In [3]:
spark=SparkSession.builder .appName("sparksql").\
    getOrCreate()

In [4]:
#CHEKING spark is running now
print(spark.version)

3.2.0


In [5]:
#creating a data frame in a scv file
data =spark.read.format('csv').\
    option('inferschema','true').\
    option('header','true').\
    option('path','operations_management.csv').\
    load()


In [6]:
#to see the schema
data.printSchema()

root
 |-- description: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- size: string (nullable = true)
 |-- line_code: string (nullable = true)
 |-- value: integer (nullable = true)



In [7]:
data.show(5)

+--------------------+--------------------+-----+---------------+---------+-----+
|         description|            industry|level|           size|line_code|value|
+--------------------+--------------------+-----+---------------+---------+-----+
|Awareness of clim...|               total|    0| 6–19 employees| C0300.01|13080|
|Awareness of clim...|               total|    0|20–49 employees| C0300.01| 3348|
|Awareness of clim...|               total|    0|50–99 employees| C0300.01| 1089|
|Awareness of clim...|               total|    0| 100+ employees| C0300.01| 1023|
|Awareness of clim...|Agriculture, fore...|    1|          total| C0300.01| 2364|
+--------------------+--------------------+-----+---------------+---------+-----+
only showing top 5 rows



In [10]:
#transforationg the data
from pyspark.sql.functions import col, desc
data_2=data.select("industry","value").\
    filter((col("value")>10000)).\
  orderBy(desc("value"))

In [11]:
data_3 = data.select("industry","value").\
filter((col("value") > 200) & (col("industry") != "total")).\
orderBy(desc("value"))

In [12]:
data_2.show(5)

+--------+-----+
|industry|value|
+--------+-----+
|   total|41091|
|   total|40431|
|   total|33984|
|   total|33750|
|   total|32652|
+--------+-----+
only showing top 5 rows



In [13]:
data_3.show(5)

+--------------------+-----+
|            industry|value|
+--------------------+-----+
|        Construction| 6030|
|        Construction| 5904|
|        Construction| 5229|
|Accommodation & f...| 5058|
|        Construction| 4965|
+--------------------+-----+
only showing top 5 rows



In [14]:
#creating a temporatry view 
data_2.createOrReplaceTempView("data")

In [15]:
spark.sql('select industry,value from data where value > 200').show(5)

+--------+-----+
|industry|value|
+--------+-----+
|   total|41091|
|   total|40431|
|   total|33984|
|   total|33750|
|   total|32652|
+--------+-----+
only showing top 5 rows



In [16]:
#creating the view
data.createOrReplaceTempView("test")

In [18]:
#selecting and showing the data  from the view 
data4=spark.sql("""select * from  test """)
data4.show(5)


+--------------------+--------------------+-----+---------------+---------+-----+
|         description|            industry|level|           size|line_code|value|
+--------------------+--------------------+-----+---------------+---------+-----+
|Awareness of clim...|               total|    0| 6–19 employees| C0300.01|13080|
|Awareness of clim...|               total|    0|20–49 employees| C0300.01| 3348|
|Awareness of clim...|               total|    0|50–99 employees| C0300.01| 1089|
|Awareness of clim...|               total|    0| 100+ employees| C0300.01| 1023|
|Awareness of clim...|Agriculture, fore...|    1|          total| C0300.01| 2364|
+--------------------+--------------------+-----+---------------+---------+-----+
only showing top 5 rows



In [20]:
#cheking the metadata
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='file:/home/christine/Desktop/spar/spark-warehouse')]

In [22]:
#listing the ables
spark.catalog.listTables()

[Table(name='data', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='test', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [23]:
spark.catalog.dropTempView("test")

In [24]:
#listing the ables
spark.catalog.listTables()

[Table(name='data', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]